In [12]:
import pandas as pd
from selenium import webdriver
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import os 
from selenium.webdriver.common.keys import Keys
from time import sleep
from datetime import datetime

In [14]:
start_time = datetime.now()
filepath = r"C:/workspace/formation/PlusSimple/Scraping/KGS/PlusSimple_ref_2000_4000.xlsx"
sheetname = "to_scrap"

df = pd.read_excel(filepath,sheet_name=sheetname)
#converting Orias codes to list
df['orias code'] = [str(x).zfill(8) for x in df['orias code']]
oriascodeList = df['orias code'].tolist()
notfound=[]

#chrome driver
driver = webdriver.Chrome(r"C:\Users\abachirahltaleb\chrome_for_scraping\chromedriver_win32\chromedriver.exe")

#Output dataframe
data=pd.DataFrame(columns=['Code ORIAS','Siren','Immatriculation','Code NAF','Adresse 1','Adresse 2','Adresse 3'])

#loop through orias codes in input excel
for lst in oriascodeList:
    
    try:
        print(lst)
        driver.get("https://www.orias.fr/")
        search = driver.find_element_by_xpath("//input[@id='searchInput']")
        search.clear()
        search.send_keys(lst)
        search.send_keys(Keys.RETURN) 
        sleep(0.2)
        wait = WebDriverWait(driver, 10)

        # orias code variable
        oriascode=lst

        # fetching hyperlinks to intermediaries
        elements = driver.find_elements_by_xpath("//table[@class = 'table table-condensed table-striped table-bordered']//tbody/tr/td[3]//a") 
        if(len(elements)==0):
            notfound.append(lst)
        Hlinks = [elem.get_attribute('href') for elem in elements]
        driver.get(Hlinks[0])
        details=driver.find_element_by_xpath('//*[@class="roundedBox span12"]').text.split("\n")

        #siren_loc = details.index('Siren')
        siren_loc = [idx for idx, s in enumerate(details) if 'Siren' in s][0]
        #codeNAF_loc = details.index('Code NAF')
        codeNAF_loc = [idx for idx, s in enumerate(details) if 'Code NAF' in s][0]
        #Immatriculation_loc = details.index('Numéro d\'immatriculation')
        Immatriculation_loc = [idx for idx, s in enumerate(details) if 'Numéro d\'immatriculation' in s][0]

        # siren, codeNAF, Immatriculation variables
        siren_val = details[siren_loc+1]
        codeNAF_val = details[codeNAF_loc+1]
        Immatriculation_val = details[Immatriculation_loc+1]

        # address variables                                    
        address=driver.find_element_by_xpath('//*[@id="intermediaryCoordinate2"]').text.split("\n")
        if(len(address)== 1): 
            addr1=""
            addr2=""
            addr3=""
        elif(len(address)==2) :   
            addr1=address[1]
            addr2=""
            addr3=""
        elif(len(address)==3) :   
            addr1=address[1]
            addr2=address[2]
            addr3=""
        else:   
            addr1=address[1]
            addr2=address[2]
            addr3=address[3]

        data = data.append({'Code ORIAS':oriascode,'Siren':siren_val,'Immatriculation':Immatriculation_val,
                              'Code NAF':codeNAF_val,'Adresse 1':addr1,'Adresse 2':addr2,'Adresse 3':addr3}, ignore_index=True)


    except:
        continue

driver.close()
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


07011185
07011964
07013578
07020076
07022017
10056492
18008339
07001839
07021847
14005793
17004636
17003919
07007137
07005112
14002220
07010617
20003791
08040597
07011219
18000895
19001816
07011009
07003796
12067582
16002297
17007362
12064806
18006138
17007018
85162361
10055929
15005664
16003723
08042499
17001894
07010869
07000126
07010992
14005369
13002013
14000992
15004644
19002567
07013391
10058381
07004838
10057949
15006013
07031061
07002004
11063150
14003624
12069128
07020677
19003894
07007802
15005899
14007327
07001210
07036452
12069146
16006489
15004082
09047249
13009784
09047024
10058781
07002314
07003445
12066580
09046844
17003499
17004811
07007306
16001973
17006474
21002515
07005636
18007882
17005734
10058104
15004627
16006404
13004284
15000091
15006010
19001705
15005389
07020844
07003059
07004866
07034310
08039963
08040330
11059199
15003755
07005191
14000897
07009262
07011210
14001182
11059927
13008987
13009645
16006909
20002886
13009857
10054734
19006460
07011944
07015371
0

07002792
07004732
07004842
07006107
07007441
07016083
13008076
15006791
16001720
16004042
16006824
07000431
07004038
07004851
07004993
07006024
07006367
07009205
07010810
07015153
07015163
07015669
09049093
10055966
10057041
11059622
11062828
12068260
12069121
13006447
15005122
15006799
17003356
18000398
18002099
18002629
18002937
18005546
18007208
19004565
20005250
07003632
13007432
07004086
15003286
15005838
07032753
14005987
09049715
07003298
07013850
12068152
13005413
15001779
17002462
20002011
07015699
08040191
13007668
20002653
07003227
07009550
20003551
17000399
14002177
08040154
11063991
07010357
07010751
20004056
07019134
07009817
07009542
14004912
07000752
11061569
12066030
07013360
19000894
17007270
09049612
20001210
09049772
07012470
07021967
15003635
07004834
07021993
07022229
09051917
07031107
20001973
20000794
10056862
17001647
14007263
10003295
20003046
20006439
11061382
15001390
07024309
13003349
15006383
19001627
19004166
07015406
07030203
15006186
18004606
18008291
1

09050102
07001948
07003364
08046335
09050518
14002218
14007152
18001285
17003976
19006986
13009094
07010474
09046827
11060139
13005626
18005550
07015036
19007803
07000001
07000025
07000106
07000108
07000148
07000158
07000197
07000218
07000248
07000256
07000257
07000263
07000305
07000327
07000340
07000370
07000387
07000405
07000417
07000441
07000448
07000459
07000477
07000494
07000505
07000556
07000594
07000601
07000608
07000609
07000612
07000637
07000646
07000676
07000682
07000699
07000750
07000789
07000812
07000832
07000840
07000870
07000871
07000874
07000880
07000884
07000982
07001031
07001036
07001047
07001058
07001094
07001200
07001273
07001305
07001322
07001331
07001391
07001493
07001542
07001545
07001549
07001578
07001589
07001632
07001634
07001636
07001677
07001701
07001717
07001725
07001747
07001754
07001822
07001854
07001859
07001894
07001989
07002017
07002034
07002085
07002128
07002144
07002158
07002160
07002188
07002191
07002192
07002209
07002216
07002238
07002239
07002246
0

In [16]:
data.to_excel(r'C:/workspace/formation/PlusSimple/Scraping/KGS/NotFound_PlusSimple_2000_4000.xlsx')

In [15]:
data

,Code ORIAS,Siren,Immatriculation,Code NAF,Adresse 1,Adresse 2,Adresse 3
0,07011185,334447463,07011185,Activités des agents et courtiers d'assurances...,17 PLACE PATTON,BP 418,50300 AVRANCHES
1,07011964,334509148,07011964,Activités des agents et courtiers d'assurances...,13 AVE DE LA LIBERATION,bp 49,56702 HENNEBONT CEDEX
2,07013578,438666125,07013578,Activités des agents et courtiers d'assurances...,9 bd Zucarelli,20200 BASTIA,
3,07020076,443750435,07020076,Activités des agents et courtiers d'assurances...,5 RUE VELOUTERIE,84000 AVIGNON,
4,07022017,453075822,07022017,Activités des agents et courtiers d'assurances...,43 GRAND RUE,68000 COLMAR,
...,...,...,...,...,...,...,...
1988,07003558,325731446,07003558,Activités des agents et courtiers d'assurances...,15 RUE JULIETTE DODU,BP 176,97464 ST DENIS
1989,07003584,439066754,07003584,Activités des agents et courtiers d'assurances...,128 AVENUE CHARLES BOUTET,BP 285,08103 CHARLEVILLE MEZIERES CEDEX
1990,07003677,439541558,07003677,Activités des agents et courtiers d'assurances...,7 rue Saint Charles,06160 JUAN LES PINS,
1991,07003717,452076086,07003717,Activités des agents et courtiers d'assurances...,3 AVENUE LOUIS LAMOTHE,BP 17,33510 ANDERNOS LES BAINS
